## MLFlow example with  tracking uri

In [16]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [17]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1200, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([1080,  120]))

In [18]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [19]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

           0       0.95      1.00      0.98       324
           1       1.00      0.56      0.71        36

    accuracy                           0.96       360
   macro avg       0.98      0.78      0.85       360
weighted avg       0.96      0.96      0.95       360



In [20]:
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_dict


{'0': {'precision': 0.9529411764705882,
  'recall': 1.0,
  'f1-score': 0.9759036144578312,
  'support': 324},
 '1': {'precision': 1.0,
  'recall': 0.5555555555555556,
  'f1-score': 0.7142857142857143,
  'support': 36},
 'accuracy': 0.9555555555555556,
 'macro avg': {'precision': 0.9764705882352941,
  'recall': 0.7777777777777778,
  'f1-score': 0.8450946643717727,
  'support': 360},
 'weighted avg': {'precision': 0.9576470588235294,
  'recall': 0.9555555555555556,
  'f1-score': 0.9497418244406196,
  'support': 360}}

In [21]:
report_dict['accuracy']

0.9555555555555556

In [ ]:
#Start the MLflow Server
import subprocess

mlflow_process = subprocess.Popen(
    ["mlflow", "server", "--host", "127.0.0.1", "--port", "5000"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)
print(f"MLflow server is running with PID: {mlflow_process.pid}")

# Run a local Tracking Server

In [22]:
# After starting the server, interact with it
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
print("MLflow tracking URI set to http://127.0.0.1:5000")

In [28]:
mlflow.set_experiment("First experiment")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'recall_class_0': report_dict['0']['recall'],
        'recall_class_1': report_dict['1']['recall'],
        'f1_score_macro': report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr, "Logistic Regression")  


2024/12/24 00:27:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run magnificent-dove-71 at: http://127.0.0.1:5000/#/experiments/980259606206388124/runs/6db61e940f6546708b6e3d7f62fd4e09
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/980259606206388124
